In [11]:
import pandas as pd
import numpy as np
import re
import timeit

import pickle

from eunjeon import Mecab
mecab = Mecab()

with open('word_dict.pkl','rb') as f:
    stopwords = pickle.load(f)
    
company = pd.read_csv('./raw/19년 1월 2일 기준(코스피).csv',encoding='utf-8')
company_list = company.iloc[:100,]['종목명'][0:100]# 시총 상위 100개 기업 리스트
total_df = pd.DataFrame(columns=['Company','Day','Main','Head'])

# 분석 중에 불용어 사전에 추가하고 업로드하기 힘들 경우, 아래와 같이 리스트를 추가해서 사용
list_del= ['기사','기자','신문','네이버','뉴스','연합뉴스','하이닉스','텔레콤','사진','데일리','가운데','구독',\
            '이날','이후','지난해','최근','삼성','종목','한국','거래소','현대','종합','주요'] + company_list.to_list()
stopwords += list_del

In [12]:
for company in company_list:
    df = pd.read_csv("./전처리/1차 전처리 기사/%s_전처리.csv" %(company),encoding='utf-8')
    df.dropna(axis=0,inplace=True)
    df['Day']=df['Day'].str[:10]
    df_head = df.groupby(['Day'])['Head_after'].apply(' '.join).reset_index()
    df_main = df.groupby(['Day'])['Main_after'].apply(' '.join).reset_index()
    df_total = pd.merge(df_head,df_main,on='Day')
    temp_pd = pd.DataFrame()
    print("------"+ company +"------")

    
    start_time = timeit.default_timer()

    main = df_total['Main_after']
    temp_list_main = []
    for i, news in enumerate(main):
        nouns_list = mecab.nouns('%s' %news) # mecab에서 nouns라는 명사 추출하는 함수 사용
        clean_words = []
        for word in nouns_list:
            if len(word) < 2:  # 단어가 한글자인 경우 생략
                continue
            if word not in stopwords: # 단어가 불용어 사전에 없을 경우만 리스트에 삽입
                clean_words.append(word)
        total_nouns = ' '.join(clean_words) # 각 기업별 하루치에 기사들을 전부 하나의 문자열로 만들어준다
        temp_list_main.append(total_nouns)
    terminate_time = timeit.default_timer()
    print("Main 처리 시간 : %f초" % (terminate_time - start_time))

    start_time = timeit.default_timer()

    head = df_total['Head_after']
    temp_list_head = []
    for i, news in enumerate(head):
        nouns_list = mecab.nouns('%s' %news)
        clean_words = []
        for word in nouns_list:
            if len(word) < 2:
                continue
            if word not in stopwords:
                clean_words.append(word)
        total_nouns = ' '.join(clean_words) 
        temp_list_head.append(total_nouns)
    terminate_time = timeit.default_timer()
    print("Head 처리 시간 : %f초" % (terminate_time - start_time))

    temp_pd['Main'] = temp_list_main
    temp_pd['Head'] = temp_list_head
    df2 = df_total.loc[:,['Day']]
    new_df = pd.concat([df2, temp_pd], axis = 1)
    total_df = pd.concat([total_df,new_df])
    total_df.fillna('%s' %(company),inplace=True)
total_df.to_csv("./전처리/2차 전처리(토큰화)/total_df.csv" , mode='w',encoding = 'utf-8-sig')

------삼성전자------
Main 처리 시간 : 269.128495초
Head 처리 시간 : 6.082477초
------SK하이닉스------
Main 처리 시간 : 66.735493초
Head 처리 시간 : 1.505375초
------셀트리온------
Main 처리 시간 : 33.565210초
Head 처리 시간 : 0.902345초
------삼성전자우------
Main 처리 시간 : 4.234805초
Head 처리 시간 : 0.127669초
------삼성바이오로직스------
Main 처리 시간 : 27.967528초
Head 처리 시간 : 0.637712초
------현대차------
Main 처리 시간 : 112.090035초
Head 처리 시간 : 2.466265초
------LG화학------
Main 처리 시간 : 38.268908초
Head 처리 시간 : 0.906689초
------SK텔레콤------
Main 처리 시간 : 75.266141초
Head 처리 시간 : 1.585973초
------한국전력------
Main 처리 시간 : 35.709490초
Head 처리 시간 : 0.870181초
------POSCO------
Main 처리 시간 : 8.740990초
Head 처리 시간 : 0.244863초
------삼성물산------
Main 처리 시간 : 24.095614초
Head 처리 시간 : 0.470484초
------NAVER------
Main 처리 시간 : 23.985648초
Head 처리 시간 : 0.527955초
------KB금융------
Main 처리 시간 : 18.150755초
Head 처리 시간 : 0.363912초
------신한지주------
Main 처리 시간 : 16.026891초
Head 처리 시간 : 0.394651초
------현대모비스------
Main 처리 시간 : 24.814500초
Head 처리 시간 : 0.615721초
------SK------
Main 처리 시간 : 31

In [15]:
total_df = pd.read_csv('./전처리/2차 전처리(토큰화)/total_df.csv')
total_df.dropna(axis=0,inplace=True)

In [23]:
# TF-IDF를 이용하기 위한 라이브러리
from sklearn.feature_extraction.text import TfidfVectorizer


list_features =[500,1000,1500,2000]
for i in list_features:
    start_time = timeit.default_timer()

    text = total_df['Head'].to_list()
# min_df : 최소한의 문서에서 나타나야할 빈도
# max_features : TF-IDF 중 피쳐의 최대 갯수 설정
    tfidf_vectorizer = TfidfVectorizer(min_df = 300, max_features = i)

    tfidfv = tfidf_vectorizer.fit(text) # 벡터라이저가 단어 학습
    final_df = pd.DataFrame(tfidfv.transform(text).toarray(), columns=sorted(tfidf_vectorizer.vocabulary_))
    df3 = total_df.loc[:,['Company','Day']].reset_index().drop(['index'],axis=1)
    df4 = pd.concat([df3, final_df], axis = 1)

    terminate_time = timeit.default_timer()
    print("TF-IDF 처리 시간 : %f초" % (terminate_time - start_time))

    df4.to_csv("./전처리/TF-IDF/head_tf-idf_%s.csv" %(str(i)) , mode='w',encoding = 'utf-8-sig')

TF-IDF 처리 시간 : 7.620674초
TF-IDF 처리 시간 : 7.750580초
TF-IDF 처리 시간 : 8.082940초
TF-IDF 처리 시간 : 8.187649초


In [21]:
list_features =[500,1000,1500,2000]
for i in list_features:
    start_time = timeit.default_timer()

    text = total_df['Main'].to_list()

    tfidf_vectorizer = TfidfVectorizer(min_df = 300, max_features = i)

    tfidfv = tfidf_vectorizer.fit(text) # 벡터라이저가 단어들을 학습합니다.
    # tfidf_vectorizer.vocabulary_ # 벡터라이저가 학습한 단어사전을 출력합니다. 
    final_df = pd.DataFrame(tfidfv.transform(text).toarray(), columns=sorted(tfidf_vectorizer.vocabulary_))
    df3 = total_df.loc[:,['Company','Day']].reset_index().drop(['index'],axis=1)
    df4 = pd.concat([df3, final_df], axis = 1)

    terminate_time = timeit.default_timer()
    print("TF-IDF 처리 시간 : %f초" % (terminate_time - start_time))

    df4.to_csv("./전처리/TF-IDF/main_tf-idf_%s.csv" %(str(i)) , mode='w',encoding = 'utf-8-sig')

TF-IDF 처리 시간 : 234.168199초
TF-IDF 처리 시간 : 239.237266초
TF-IDF 처리 시간 : 245.417567초
TF-IDF 처리 시간 : 249.618352초
